In [1]:
#핵심 라이브러리 임포트

In [2]:
import torch #PyTorch의 핵심 라이브러리로, 텐서 연산, 자동 미분, 신경망 등을 다룹니다.
import torch.nn as nn #신경망을 구성하는데 필요한 다양한 모듈을 제공하는 라이브러리입니다. 여기서는 신경망 계층과 활성화 함수 등을 사용합니다.
import torch.optim as optim #모델을 학습시키기 위한 최적화 알고리즘을 포함하고 있습니다.
import torchvision #이미지 처리 및 컴퓨터 비전과 관련된 도구와 데이터셋을 제공합니다.
import torchvision.transforms as transforms #이미지 데이터에 대한 전처리(transformations)를 도와주는 모듈입니다. 여기서는 이미지를 텐서(tensor)로 변환하고, 정규화하는 작업에 사용됩니다.

In [3]:
#데이터셋 로드 및 전처리
transform = transforms.Compose([ 
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
#transforms.Compose: 여러 전처리 작업을 순차적으로 적용할 수 있도록 묶어주는 역할을 합니다.
#ToTensor(): 이미지를 PyTorch 텐서(tensor)로 변환합니다. 텐서는 PyTorch에서 데이터를 다루는 기본 단위입니다.
#Normalize((0.5,), (0.5,)): 데이터를 평균 0.5, 표준편차 0.5로 정규화(normalization)합니다. 이는 모델이 각 픽셀 값을 더 쉽게 학습하도록 돕습니다.

In [4]:
# MNIST 데이터셋 로드
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

#datasets.MNIST: MNIST 데이터셋을 가져옵니다. train=True는 학습용 데이터를 가져온다는 의미이며, download=True는 데이터셋이 없을 경우 자동으로 다운로드하게 합니다.
#DataLoader: 데이터를 배치(batch)로 나누어 모델에 입력하고, 무작위로 섞어주는 역할을 합니다. 배치 크기는 64이며, shuffle=True는 학습 데이터의 순서를 무작위로 섞습니다.


In [5]:
#신경망 모델 정의
class SimpleANN(nn.Module):
    def __init__(self):
        super(SimpleANN, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)  # 입력층에서 은닉층으로
        self.fc2 = nn.Linear(128, 64)       # 은닉층에서 은닉층으로
        self.fc3 = nn.Linear(64, 10)        # 은닉층에서 출력층으로

    def forward(self, x):
        x = x.view(-1, 28 * 28)  # 입력 이미지를 1차원 벡터로 변환
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

#SimpleANN: 이 클래스는 간단한 인공 신경망(ANN)을 정의합니다. fc1, fc2, fc3는 각각의 선형 계층(fully connected layers)을 나타냅니다.
#fc1: 28 x 28 크기의 이미지를 펼쳐서 128개의 노드로 보내는 입력 계층입니다.
#fc2: 128개의 노드를 받아 64개의 노드로 출력하는 은닉 계층입니다.
#fc3: 64개의 노드를 받아 10개의 출력 노드로 변환하는 출력 계층입니다. (MNIST는 10개의 숫자(0~9)를 분류하므로 10개의 출력이 필요합니다.)
#forward: 입력 데이터가 신경망을 통과하는 과정(순전파)을 정의합니다. 이미지를 1차원 벡터로 변환한 후, ReLU 활성화 함수를 거쳐 신경망을 통과합니다.


In [6]:
#모델 초기화 및 성능 평가(학습 전)
model = SimpleANN()

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')

#torch.no_grad(): 모델이 학습되지 않도록 하는 구문입니다. 테스트 데이터로 평가할 때는 역전파나 가중치 업데이트가 필요 없으므로 사용됩니다.
#torch.max: 신경망이 예측한 결과 중 가장 높은 값을 가진 클래스를 선택합니다.
#correct와 total: 정확히 예측한 개수와 전체 데이터를 계산하여, 정확도를 출력합니다.


Accuracy of the network on the 10000 test images: 6.62%


In [7]:
#손실 함수와 최적화 알고리즘 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

#CrossEntropyLoss: 분류 문제에서 사용되는 손실 함수입니다. 모델의 출력과 실제 정답의 차이를 계산합니다.
#SGD: 확률적 경사 하강법(Stochastic Gradient Descent) 최적화 알고리즘입니다. lr=0.01은 학습률, momentum=0.9은 속도를 높이는 매개변수입니다.

In [8]:
#모델 학습
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0
print('Finished Training')
#epoch: 전체 데이터셋을 한 번 학습하는 것을 1 에포크라고 하며, 여기서는 10 에포크 동안 학습합니다.
#optimizer.zero_grad(): 이전 학습에서 계산된 기울기를 초기화합니다.
#loss.backward(): 역전파를 통해 기울기를 계산합니다.
#optimizer.step(): 계산된 기울기를 기반으로 가중치를 업데이트합니다.
#running_loss: 배치마다 손실을 기록하여 출력합니다.

[Epoch 1, Batch 100] loss: 1.435
[Epoch 1, Batch 200] loss: 0.503
[Epoch 1, Batch 300] loss: 0.391
[Epoch 1, Batch 400] loss: 0.340
[Epoch 1, Batch 500] loss: 0.328
[Epoch 1, Batch 600] loss: 0.305
[Epoch 1, Batch 700] loss: 0.265
[Epoch 1, Batch 800] loss: 0.258
[Epoch 1, Batch 900] loss: 0.252
[Epoch 2, Batch 100] loss: 0.213
[Epoch 2, Batch 200] loss: 0.182
[Epoch 2, Batch 300] loss: 0.186
[Epoch 2, Batch 400] loss: 0.187
[Epoch 2, Batch 500] loss: 0.183
[Epoch 2, Batch 600] loss: 0.182
[Epoch 2, Batch 700] loss: 0.171
[Epoch 2, Batch 800] loss: 0.154
[Epoch 2, Batch 900] loss: 0.172
[Epoch 3, Batch 100] loss: 0.136
[Epoch 3, Batch 200] loss: 0.145
[Epoch 3, Batch 300] loss: 0.124
[Epoch 3, Batch 400] loss: 0.145
[Epoch 3, Batch 500] loss: 0.134
[Epoch 3, Batch 600] loss: 0.145
[Epoch 3, Batch 700] loss: 0.136
[Epoch 3, Batch 800] loss: 0.136
[Epoch 3, Batch 900] loss: 0.120
[Epoch 4, Batch 100] loss: 0.110
[Epoch 4, Batch 200] loss: 0.109
[Epoch 4, Batch 300] loss: 0.111
[Epoch 4, 

In [9]:
#최종 모델 평가
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')

Accuracy of the network on the 10000 test images: 97.47%
